In [8]:
# --- CELDA 1: CONFIGURACIÓN E INSTALACIÓN ---
# Forzamos actualización para arreglar el error de 'MessageFactory' (protobuf)
!pip install -U -q transformers datasets accelerate

# Desactivamos WANDB antes de importar nada para evitar que pida login o falle
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1" # Ayuda a depurar si falla la GPU

print("✅ Librerías instaladas y entorno configurado.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 90.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.6 MB/s eta 0:00:00:00:01
✅ Librerías instaladas y entorno configurado.


In [12]:
!pip install -U -q transformers datasets accelerate

import os
# Desactivar reporte a la nube para evitar bloqueos
os.environ["WANDB_DISABLED"] = "true"
print("✅ Celda 1: Librerías instaladas.")

✅ Celda 1: Librerías instaladas.


In [13]:
import os

dataset_path = "/kaggle/input/dataset-fusion-v3"
model_zip_path = "/kaggle/input/modelo-ascii-art-v3"

print("🔍 Buscando archivos en /kaggle/input...")

for root, dirs, files in os.walk("/kaggle/input"):
    for file in files:
        # Busca el txt del dataset
        if file.endswith("READY.txt"):
            dataset_path = os.path.join(root, file)
            print(f"   📄 Dataset encontrado: {dataset_path}")
        
        # Busca el zip del modelo
        elif file.endswith(".zip") and "modelo" in file:
            model_zip_path = os.path.join(root, file)
            print(f"   🧠 Modelo ZIP encontrado: {model_zip_path}")

if not dataset_path or not model_zip_path:
    print("❌ ERROR CRÍTICO: No encontré los archivos. Revisa la pestaña 'Input' a la derecha.")
else:
    print("✅ Celda 2: Archivos localizados y listos.")

🔍 Buscando archivos en /kaggle/input...
   📄 Dataset encontrado: /kaggle/input/dataset-fusion-v3/dataset_FINAL_V3_READY.txt
✅ Celda 2: Archivos localizados y listos.


In [14]:
import zipfile

extract_path = "/kaggle/working/modelo_base"

if not os.path.exists(extract_path):
    print(f"📦 Descomprimiendo modelo en {extract_path}...")
    try:
        with zipfile.ZipFile(model_zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print("✅ Celda 3: Modelo descomprimido correctamente.")
    except Exception as e:
        print(f"❌ Error al descomprimir: {e}")
else:
    print("ℹ️ El modelo ya estaba descomprimido.")

📦 Descomprimiendo modelo en /kaggle/working/modelo_base...
❌ Error al descomprimir: [Errno 21] Is a directory: '/kaggle/input/modelo-ascii-art-v3'


In [15]:
# --- CELDA MAESTRA: LOCALIZADOR INTELIGENTE (Versión Kaggle Dataset) ---
import os
from transformers import GPT2LMHeadModel, GPT2Tokenizer

dataset_path = None
model_folder_path = None

print("🔍 Escaneando Kaggle Input en busca del cerebro y el libro...")

# Recorremos todo para encontrar el .txt y la carpeta del modelo
for root, dirs, files in os.walk("/kaggle/input"):
    # 1. BUSCAR EL DATASET DE TEXTO
    for file in files:
        if "READY.txt" in file:
            dataset_path = os.path.join(root, file)
            print(f"   📄 Dataset encontrado: {dataset_path}")

    # 2. BUSCAR LA CARPETA DEL MODELO (Buscamos 'config.json' para confirmar)
    for file in files:
        if file == "config.json":
            # Si encontramos config.json, ESTA es la carpeta del modelo
            model_folder_path = root
            print(f"   🧠 Carpeta del Modelo encontrada: {model_folder_path}")

if not dataset_path:
    print("❌ ERROR: No encuentro el archivo .txt (READY).")
elif not model_folder_path:
    print("❌ ERROR: No encuentro la carpeta del modelo (config.json).")
else:
    print("✅ ¡Archivos localizados!")
    print("   (Kaggle ya los descomprimió, así que usaremos la carpeta directa)")

    # --- CARGA DIRECTA DEL MODELO ---
    print("\n⚙️ Cargando modelo desde la carpeta detectada...")
    try:
        tokenizer = GPT2Tokenizer.from_pretrained(model_folder_path)
        model = GPT2LMHeadModel.from_pretrained(model_folder_path)
        tokenizer.pad_token = tokenizer.eos_token
        print("✅ MODELO CARGADO EXITOSAMENTE.")
        
        # Variable clave para las siguientes celdas
        extract_path = model_folder_path 
        
    except Exception as e:
        print(f"❌ Error al cargar: {e}")

🔍 Escaneando Kaggle Input en busca del cerebro y el libro...
   📄 Dataset encontrado: /kaggle/input/dataset-fusion-v3/dataset_FINAL_V3_READY.txt
   🧠 Carpeta del Modelo encontrada: /kaggle/input/modelo-ascii-art-v3
✅ ¡Archivos localizados!
   (Kaggle ya los descomprimió, así que usaremos la carpeta directa)

⚙️ Cargando modelo desde la carpeta detectada...
✅ MODELO CARGADO EXITOSAMENTE.


In [16]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

print("⚙️ Cargando Modelo y Tokenizer...")
try:
    tokenizer = GPT2Tokenizer.from_pretrained(extract_path)
    model = GPT2LMHeadModel.from_pretrained(extract_path)
    
    # Asegurar token de pad
    tokenizer.pad_token = tokenizer.eos_token
    print("✅ Celda 4: Modelo cargado en memoria.")
except Exception as e:
    print(f"❌ Error cargando modelo: {e}")
    print("Verifica que la Celda 3 haya terminado bien.")

⚙️ Cargando Modelo y Tokenizer...
✅ Celda 4: Modelo cargado en memoria.


In [19]:
# --- CELDA MAESTRA: LOCALIZADOR INTELIGENTE (Versión Kaggle Dataset) ---
import os
from transformers import GPT2LMHeadModel, GPT2Tokenizer

dataset_path = None
model_folder_path = None

print("🔍 Escaneando Kaggle Input en busca del cerebro y el libro...")

# Recorremos todo para encontrar el .txt y la carpeta del modelo
for root, dirs, files in os.walk("/kaggle/input"):
    # 1. BUSCAR EL DATASET DE TEXTO
    for file in files:
        if "READY.txt" in file:
            dataset_path = os.path.join(root, file)
            print(f"   📄 Dataset encontrado: {dataset_path}")

    # 2. BUSCAR LA CARPETA DEL MODELO (Buscamos 'config.json' para confirmar)
    for file in files:
        if file == "config.json":
            # Si encontramos config.json, ESTA es la carpeta del modelo
            model_folder_path = root
            print(f"   🧠 Carpeta del Modelo encontrada: {model_folder_path}")

if not dataset_path:
    print("❌ ERROR: No encuentro el archivo .txt (READY).")
elif not model_folder_path:
    print("❌ ERROR: No encuentro la carpeta del modelo (config.json).")
else:
    print("✅ ¡Archivos localizados!")
    print("   (Kaggle ya los descomprimió, así que usaremos la carpeta directa)")

    # --- CARGA DIRECTA DEL MODELO ---
    print("\n⚙️ Cargando modelo desde la carpeta detectada...")
    try:
        tokenizer = GPT2Tokenizer.from_pretrained(model_folder_path)
        model = GPT2LMHeadModel.from_pretrained(model_folder_path)
        tokenizer.pad_token = tokenizer.eos_token
        print("✅ MODELO CARGADO EXITOSAMENTE.")
        
        # Variable clave para las siguientes celdas
        extract_path = model_folder_path 
        
    except Exception as e:
        print(f"❌ Error al cargar: {e}")

🔍 Escaneando Kaggle Input en busca del cerebro y el libro...
   📄 Dataset encontrado: /kaggle/input/dataset-fusion-v3/dataset_FINAL_V3_READY.txt
   🧠 Carpeta del Modelo encontrada: /kaggle/input/modelo-ascii-art-v3
✅ ¡Archivos localizados!
   (Kaggle ya los descomprimió, así que usaremos la carpeta directa)

⚙️ Cargando modelo desde la carpeta detectada...
✅ MODELO CARGADO EXITOSAMENTE.


In [22]:
# --- CELDA 5 CORREGIDA: COPIAR Y PROCESAR ---
import shutil
from transformers import TextDataset, DataCollatorForLanguageModeling

print("🚚 Moviendo dataset a zona segura (Writable)...")

# 1. Copiamos el archivo a /kaggle/working para poder escribir el caché sin errores
local_dataset_path = "/kaggle/working/dataset_training.txt"
shutil.copy(dataset_path, local_dataset_path)

print(f"✅ Archivo copiado a: {local_dataset_path}")
print("📚 Procesando Dataset ahora sí...")

try:
    # 2. Cargamos desde la copia local
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=local_dataset_path, # <--- Usamos la copia
        block_size=128,
        overwrite_cache=True
    )
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False
    )
    print(f"✅ Celda 5: Dataset procesado EXITO. Bloques creados: {len(train_dataset)}")
except Exception as e:
    print(f"❌ Error procesando dataset: {e}")

🚚 Moviendo dataset a zona segura (Writable)...
✅ Archivo copiado a: /kaggle/working/dataset_training.txt
📚 Procesando Dataset ahora sí...
✅ Celda 5: Dataset procesado EXITO. Bloques creados: 42863


In [23]:
from transformers import Trainer, TrainingArguments

output_dir = "/kaggle/working/ascii-gpt-v4-general"

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=4,            # 4 Épocas
    per_device_train_batch_size=8, # Buen tamaño para T4 x2
    save_steps=1000,
    save_total_limit=2,
    learning_rate=5e-5,
    fp16=True,                     # Usar aceleración mixta
    report_to="none",              # Sin reporte externo
    logging_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

print("🚀 INICIANDO ENTRENAMIENTO...")
trainer.train()
print("✅ Celda 6: Entrenamiento finalizado.")

🚀 INICIANDO ENTRENAMIENTO...


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,0.636800
200,0.629400
300,0.625700
400,0.636200
500,0.643000
600,0.608800
700,0.623200
800,0.614100
900,0.620500
1000,0.640900


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


✅ Celda 6: Entrenamiento finalizado.


In [24]:
import shutil

final_path = "/kaggle/working/modelo_final_v4"
zip_output_name = "/kaggle/working/ASCII-GPT-V4-GENERAL"

print("💾 Guardando modelo final...")
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)

print("📦 Comprimiendo ZIP...")
shutil.make_archive(zip_output_name, 'zip', final_path)

print("="*40)
print(f"🎉 ¡TODO LISTO! Descarga el archivo {zip_output_name}.zip en la sección Output.")

💾 Guardando modelo final...
📦 Comprimiendo ZIP...
🎉 ¡TODO LISTO! Descarga el archivo /kaggle/working/ASCII-GPT-V4-GENERAL.zip en la sección Output.


In [25]:
# --- CELDA 8: LA PRUEBA DE FUEGO (INFERENCIA) ---
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Cargar el modelo RECIÉN ENTRENADO
model_path = "/kaggle/working/modelo_final_v4"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)
model.to("cuda") # Mandar a GPU

def dibujar(prompt, temp=0.7):
    model.eval()
    prompt_formatted = f"{prompt}\nAI:\n"
    inputs = tokenizer.encode(prompt_formatted, return_tensors='pt').to("cuda")

    print(f"🎨 Dibujando: {prompt}...")
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=500,
            do_sample=True,
            temperature=temp,
            top_k=50,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    texto = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Limpiamos para mostrar solo el dibujo
    dibujo = texto.replace(prompt_formatted, "")
    print("="*40)
    print(texto) # Mostramos todo por si acaso
    print("="*40)

# --- PROBAR CONCEPTOS ---
dibujar("Animals Cat")
dibujar("Space Rocket")
dibujar("Weapons Sword")
dibujar("Buildings Castle")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


🎨 Dibujando: Animals Cat...
Animals Cat
AI:
                                                                              ,;, 
                                                                                ,;, 
                                                                                                                  ,;,
                                                                                                   ,;, 
                                                                                     ,;,                    
🎨 Dibujando: Space Rocket...
Space Rocket
AI:
                                                                                                                                                                                                                                                                                                                                                                                                                           

In [ ]:
### --- CELDA 9: ENTRENAMIENTO EXTENDIDO (V5 - HARDCORE) ---
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel, GPT2Tokenizer

# 1. Cargar el modelo V4 (el de la espada infinita)
print("♻️  Cargando modelo V4 para segunda ronda...")
last_model_path = "/kaggle/working/modelo_final_v4"
model = GPT2LMHeadModel.from_pretrained(last_model_path)
tokenizer = GPT2Tokenizer.from_pretrained(last_model_path)

# 2. Configuración para 2 HORAS de gimnasio
# Bajamos el Learning Rate para "pulir" detalles sin romper lo aprendido
output_dir_v5 = "/kaggle/working/ascii-gpt-v5-hardcore"

training_args = TrainingArguments(
    output_dir=output_dir_v5,
    overwrite_output_dir=True,
    num_train_epochs=8,            # 8 Épocas (aprox 1.5 - 2 horas)
    per_device_train_batch_size=8,
    save_steps=2000,
    save_total_limit=2,
    learning_rate=1e-5,            # <--- MUY BAJO (Cirugía de precisión)
    fp16=True,
    report_to="none",
    logging_steps=100
)

# Reutilizamos el data_collator y train_dataset que ya tienes en memoria
# (Si borraste variables, avísame, pero deberían seguir ahí de la celda anterior)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

print("💪 INICIANDO SESIÓN DE GIMNASIO INTENSIVO (8 ÉPOCAS)...")
print("   (Esto tomará entre 90 y 120 minutos. Ve por botanas.)")

trainer.train()

# --- GUARDADO FINAL ---
final_path_v5 = "/kaggle/working/modelo_final_v5"
print("💾 Guardando modelo V5 (El Musculoso)...")
trainer.save_model(final_path_v5)
tokenizer.save_pretrained(final_path_v5)

import shutil
shutil.make_archive("/kaggle/working/ASCII-GPT-V5-HARDCORE", 'zip', final_path_v5)
print("🎉 ¡ENTRENAMIENTO COMPLETADO! Descarga 'ASCII-GPT-V5-HARDCORE.zip'")

In [27]:
# --- CELDA 10: LA REVELACIÓN (Testeo del V5 Hardcore) ---
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# 1. Cargar al "Musculoso" (V5)
model_path = "/kaggle/working/modelo_final_v5"

print(f"📂 Cargando cerebro desde: {model_path}...")
try:
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    model = GPT2LMHeadModel.from_pretrained(model_path)
    model.to("cuda") # A la GPU
    print("✅ ¡Cerebro V5 cargado y listo!")
except Exception as e:
    print(f"❌ Error cargando: {e}")
    print("Asegúrate de que la carpeta existe en el panel de la derecha.")

# 2. Función de Dibujo
def generar_arte(prompt, temperature=0.7):
    model.eval()
    # Formato exacto del entrenamiento
    prompt_formatted = f"{prompt}\nAI:\n"
    
    input_ids = tokenizer.encode(prompt_formatted, return_tensors='pt').to("cuda")
    
    print(f"\n🎨 PROMPT: {prompt} (Temp: {temperature})")
    print("="*50)
    
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=600,        # Suficiente espacio para dibujar
            do_sample=True,        # Creatividad activada
            temperature=temperature,
            top_k=40,              # Filtrar ruido
            top_p=0.9,             # Nucleus sampling
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1 # Evitar bucles infinitos
        )
    
    ascii_art = tokenizer.decode(output[0], skip_special_tokens=True)
    # Limpiamos el prompt para ver solo el dibujo
    ascii_art = ascii_art.replace(prompt_formatted, "")
    print(ascii_art)
    print("="*50)

# --- RONDA DE PRUEBAS ---
# Probamos cosas que sabemos que existen en el dataset masivo

print("🔍 INICIANDO INSPECCIÓN VISUAL...")

# 1. Los clásicos
generar_arte("Animals Cat", temperature=0.6)
generar_arte("Buildings Castle", temperature=0.6)

# 2. Cosas complejas (Del dataset nuevo)
generar_arte("Space Rocket", temperature=0.7)
generar_arte("Weapons Sword", temperature=0.5) # Temp baja para que la espada salga recta
generar_arte("Music Guitar", temperature=0.7)

# 3. La prueba definitiva
generar_arte("Miscellanous Skull", temperature=0.8)

📂 Cargando cerebro desde: /kaggle/working/modelo_final_v5...
✅ ¡Cerebro V5 cargado y listo!
🔍 INICIANDO INSPECCIÓN VISUAL...

🎨 PROMPT: Animals Cat (Temp: 0.6)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

🎨 PROMPT: Buildings Castle (Temp: 0.6)
                                                                                                                                                                                                              

In [28]:
# --- CELDA 11: INFERENCIA FORZADA (Rompiendo el hielo) ---
def forzar_arte(prompt, temperature=0.8):
    model.eval()
    
    # TRUCO 1: Inyectamos un punto "." al inicio para obligarlo a empezar a dibujar
    # El prompt termina en "\nAI:\n."
    prompt_formatted = f"{prompt}\nAI:\n." 
    
    input_ids = tokenizer.encode(prompt_formatted, return_tensors='pt').to("cuda")
    
    print(f"🎨 PROMPT: '{prompt}' | Temp: {temperature}")
    print("-" * 40)
    
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=500,
            do_sample=True,
            temperature=temperature, # Un poco más alto para variedad
            top_k=50,
            top_p=0.92,
            
            # TRUCO 2: CASTIGO A LA REPETICIÓN
            # Esto evita que ponga 500 espacios en blanco seguidos
            repetition_penalty=1.4, 
            
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # Decodificamos
    full_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Limpieza visual
    # Quitamos el prompt para ver solo el arte
    ascii_art = full_text.split("AI:\n")[1] if "AI:\n" in full_text else full_text
    print(ascii_art)
    print("=" * 50)

# --- RONDA DE PRUEBAS (Nombres Corregidos) ---

# 1. Gatos (Plural, como en el dataset)
forzar_arte("Animals Cats", temperature=0.7)

# 2. Espadas (Plural)
forzar_arte("Weapons Swords", temperature=0.6)

# 3. Castillos (La categoría era Buildings And Places Castles)
#    Probemos con la versión corta y la larga por si acaso
forzar_arte("Buildings And Places Castles", temperature=0.7)

# 4. Algo del espacio
forzar_arte("Space Planets", temperature=0.8)

# 5. La calavera que sí funcionó (repetimos para confirmar)
forzar_arte("Miscellanous", temperature=0.8) # A veces la categoria sola funciona

🎨 PROMPT: 'Animals Cats' | Temp: 0.7
----------------------------------------
.---------------------------------.  |\| .----------------------------------,  /|/|/|/|/.|/|/|/|/|| ||| \|/|/|/.|/|/|/|/
/|/|/|/|/|./'  '-..-'.'`\.//.\.-'/.'\'-...-''''.\\/_\/_/(\/\/)\/ _(\/\/)\^/ // \\((/ ^ )\|||||||  ,---._
|__......----........._________....-----.....___...------.......___  ....---+|____ ......------....______|  ___
| _______|/ _____ _______________________________________________________|  ______  [] []
|  `-----------"""--------"------------""-------"' --[]-|  :[][]|==]][][=:] [=>]=::[ ]|==>>]=======[[[]=:::::[]=]:[]=:::::::::.]|  +>+++<++<++++<<<<><+<+++++++++++>
| ======(=========[=""===============================================================}|  =================== }==== }====== ===={
|  ---------------------------------------------------------"----------(@)"-------------------------------(_)|  ------------------------------=()
|
|  ------"------------- "---------------------------

In [29]:
# --- CELDA FINAL: RONDA DE DESPEDIDA ---
def prueba_rapida(categoria, temp=0.7):
    # Prompt formateado con el truco del punto "."
    full_prompt = f"{categoria}\nAI:\n."
    inputs = tokenizer.encode(full_prompt, return_tensors='pt').to("cuda")
    
    print(f"\n🌙 PROBANDO: {categoria}")
    with torch.no_grad():
        out = model.generate(
            inputs, 
            max_length=400, 
            do_sample=True, 
            temperature=temp,
            repetition_penalty=1.3, # Para que no se repita tanto
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    print(tokenizer.decode(out[0], skip_special_tokens=True).replace(full_prompt, ""))
    print("-" * 40)

# --- TUS DESEOS ANTES DE DORMIR ---
prueba_rapida("Animals Dogs", temp=0.6)
prueba_rapida("Computers Computers", temp=0.7)
prueba_rapida("Logos Apple", temp=0.8)
prueba_rapida("Space Aliens", temp=0.7)


🌙 PROBANDO: Animals Dogs
  .-.
`-' o
    \,
    /  `\_\__
jgs (_) (___) _)

  ___
(____)(_______)_____
|  |  ||  __||
||___/  )  ---(_)|===Lux
<____________>'==============='-----''--._
  ^^^^^^^^ ((o)))~~~---...------~~~~ ~-..______.-=~-------.....=====....--------=-.,
    ,-"""-.     ....==============;'=====.
  //  -"-"----^"""-----^ ----'-.----\-^-----------------------------".---------.
  /  --"'--.--. ..=====.;=========/     ======......-~       ""--------------.
((    '---.'                '-..=======(()))
//         :      ==========                 ````---..                                        ''--......=====::..~~~-......-------..''''--..------------..i
                       "--'-,::::::::::::......=====:'
\\   ;;;;;;;;;;;;;;;;;:::::;;;;;;;@@@a@@@@aaaaaa@@@@@@@a@@@@@@@@@@@@@@a,%%%
                
----------------------------------------

🌙 PROBANDO: Computers Computers
---------------------------------------------._  _.-'  .------'.  '-.'--..._
|_______________|  |  (o)